# Transfer Learning with TensorFlow Part 3

Scaling Up (Food Vision mini)
101 total food classes

In [ ]:
# Check gpu
!nvidia-smi

## Setup Helper Functions

In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## Food 101

Goal: Beat the original Food101 paper with 10% of the training data
### Download Dataset

In [ ]:
# Download data
!wget -nc -P ../Downloads/ https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

# Unzip
unzip_data('../Downloads/101_food_classes_10_percent.zip', '../Downloads')

# Check number of images and subdirectories in the dataset
walk_through_dir('../Downloads/101_food_classes_10_percent')

In [ ]:
train_dir = '../Downloads/101_food_classes_10_percent/train'
test_dir = '../Downloads/101_food_classes_10_percent/test'

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False) # don't shuffle test data for prediction analysis

## Training the big model with transfer learning

10% of 101 food classes

Steps:
1. Create a ModelCheckpoint callback
2. Create a data agumentation layer to build data agumentation right into the model
3. Build a headless (no top layers) functional EfficientNetB0 base model
    a. Create a custom output layer
4. Compile
5. Feature extract for 5 full passes 
    a. 5 epochs on the train dataset
    b. Validate on 15% of the test data (to save time)

In [ ]:
checkpoint_path = '../checkpoints/101_classes_10_percent_data_model_checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                        save_weights_only=True,
                                                        monitor="val_accuracy",
                                                        save_best_only=True,
                                                        save_freq="epoch", #default is save every epoch,
                                                        verbose=1
                                                        )

In [ ]:
from helper_functions import print_random_img

print_random_img(test_dir, test_data)

In [ ]:
# Create data aug layer
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data aug

with tf.device('/cpu:0'):
    data_aug = Sequential([
        preprocessing.RandomFlip('horizontal'),
        preprocessing.RandomRotation(0.2),
        preprocessing.RandomHeight(0.2),
        preprocessing.RandomWidth(0.2),
        preprocessing.RandomZoom(0.2),
    #     preprocessing.Rescaling(1/255.), # rescale inputs of images to between 0 & 1. Required for models like resnet50 
    ], name="data_aug")



In [ ]:
# Setup base model and freeze
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_aug(inputs) # augment images (only during training)
x = base_model(x, training=False) # put the  base model in inference mode so weights are not updated
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
            optimizer=tf.keras.optimizers.Adam(),
            metrics=['accuracy'])


In [ ]:
# Fit
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=5, # fit for 5 epochs to keep experiments quick
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)), # evaluate on smaller portion of test data
                                           callbacks=[checkpoint_callback]) # save best model weights to file

In [ ]:
fine_tuning_results = model.evaluate(test_data)
fine_tuning_results

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

In [ ]:
# Fine Tune
# Unfreeze all the layers in the base_model
base_model.trainable = True

# Refreeze every layer except top 5 layers
for layer in base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
# recompile model, lower learning rate by 10x
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            metrics=['accuracy'])

In [ ]:
# print layer name and index if layer is trainable
for layer_number, layer in enumerate(model.layers[2].layers):
    if layer.trainable:
        print(f'Index: {layer_number}', f'Name: {layer.name}')

In [ ]:

fine_tune_epochs = 10

#fine-tune our model
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
          epochs=fine_tune_epochs, # fit for 5 epochs to keep experiments quick
          validation_data=test_data,
          validation_steps=int(0.15 * len(test_data)), # evaluate on smaller portion of test data
          initial_epoch=history_all_classes_10_percent.epoch[-1], # start from the last epoch of the previous training
          ) # save best model weights to file

In [ ]:
feature_extraction_results = model.evaluate(test_data)

In [ ]:
compare_historys(original_history=history_all_classes_10_percent, 
                new_history=history_all_classes_10_percent_fine_tune,
                initial_epochs=5)

## Saving and loading the model

In [ ]:
# Save
model.save("../models/101_food_classes_10_percent_data_model")

In [ ]:
# Load and evaluate
loaded_model = tf.keras.models.load_model("../models/101_food_classes_10_percent_data_model")

In [ ]:
loaded_model_results = loaded_model.evaluate(test_data)
loaded_model_results

## Evaluating the performance across all different classes

1. Make predictions
2. Visualize
3. Find which predictions were the "most" wrong

In [ ]:
# Make predictions with model
preds_probs = model.predict(test_data, verbose=1)

In [ ]:
# How many predictions
len(preds_probs)

In [ ]:
# How many class predictions per image?
# What's the shape of our predictions
preds_probs.shape

In [ ]:
# check first 10 predictions
preds_probs[:10]

In [ ]:
# Check first prediction probability array
preds_probs[0]


In [ ]:
# The model outputs a prediction probability array (with N number of variables, 
# where N is the number of classes) for each sample passed to the predict method. 
# The highest probability is the predicted class.
print(f'Number of prediction probabilities for sample 0: {len(preds_probs[0])}')
print(f'Prediction probabilities for sample 0: {preds_probs[0]}')
print(f'The class with the highest predicted probability by the model for sample 0: {preds_probs[0].argmax()}')

In [ ]:
print(f'Highest prediction probability classname for sample 0: {test_data.class_names[preds_probs[0].argmax()]}')

In [ ]:
# Get the predicted classes of each label
pred_classes = preds_probs.argmax(axis=1)
pred_classes[:10]

In [ ]:
# To evaluate, we need to compare to the orginal test dataset labels
# need to unbatch test_data
y_labels = []
for images, labels in test_data.unbatch():
    y_labels.append(labels.numpy().argmax())
    
y_labels[:10]

In [ ]:
# How many y_labels are there?
len(y_labels)

## Evaluating our model's predictions

One way to check the model's predictions array is in the same order as our test labels array is to check the accuracy score.


In [ ]:
# Use scikit-learn to compare the predicted classes to the original labels
from sklearn.metrics import accuracy_score
import numpy as np

sklearn_accuracy = accuracy_score(y_true=y_labels, y_pred=pred_classes)

# Does this metric come close to our model's evaluate results?
np.isclose(loaded_model_results[1], sklearn_accuracy)

In [ ]:
# Confusion matrix
import helper_functions as hf
import importlib
importlib.reload(hf)

hf.make_confusion_matrix(y_true=y_labels, 
                        y_pred=pred_classes, 
                        classes=test_data.class_names, 
                        figsize=(100,100), 
                        text_size=30,
                        savefig=True)


## Classification Report

`classification_report`: Scikit-learn's helpful function for acquiring many different classification metrics per class. (e.g. precision, recall, f1 score, etc.)

In [ ]:
from sklearn.metrics import classification_report
print(
    classification_report(y_true=y_labels, 
                            y_pred=pred_classes, 
                            target_names=test_data.class_names
                        )
        )

In [ ]:
# Get a dictionary of the classification
classification_report_dict = classification_report(y_true=y_labels,
                            y_pred=pred_classes,
                            output_dict=True,
                            target_names=test_data.class_names
                            )

In [ ]:
# Create empty dictionary 
class_f1_scores = {}
# Loop through each dictionary item and assign name to f1-score in separate dictionary
for k, v in classification_report_dict.items():
    if k == 'accuracy': # stop loop on 'accuracy'
        break;
    else:
        # Add class names and f1-scores to new dictionary
        class_f1_scores[k] = v["f1-score"]

class_f1_scores
# classification_report_dict["bibimbap"]["f1-score"]


In [ ]:
import pandas as pd 
f1_scores = pd.DataFrame({"class_names":list(class_f1_scores.keys()),
                         "f1-score":list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)

f1_scores[:10]

In [ ]:
# plot on bar chart

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values) # get f1-score values
ax.set_yticks(range(len(f1_scores))) # set y-axis ticks to match f1-score values
ax.set_yticklabels(f1_scores["class_names"]) # set y-axis tick labels to match class names
ax.set_xlabel("F1-score")
ax.set_title("F1-score for 101 Food Classes")
ax.invert_yaxis(); # reverse order

ax.bar_label(scores, fmt='%.2f')
ax.set_xlim(right=1.05)  # adjust xlim to fit labels

### Visualizing Predictions on Custom Images

Visualize our model's predictions on our own images
Create a function to:
1. Load an image
2. Resize it to the appropriate size
3. Preprocess it
4. Predict the image
5. Visualize the image and the prediction


In [ ]:
# Create a function to load and prepare images
def load_and_prep_images(filename, img_shape=224, scale=True):
    """
    Reads in an image from filename, turns it into a tensor and reshapes into specified shape 
    (img_shape, img_shape, color_channels=3).

    Args:
        filename (str): path to target image
        image_shape (int): height/width dimension of target image size
        scale (bool): scale pixel values from 0-255 to 0-1 or not

    Returns:
        Image tensor of shape (img_shape, img_shape, 3)
    """

    # Read in image
    img = tf.io.read_file(filename)
    # Decode image into a tensor
    img = tf.image.decode_image(img, channels=3)
    # Resize image to specified shape
    img = tf.image.resize(img, [img_shape, img_shape])
    # Scale pixel values from 0-255 to 0-1
    if scale:
        img = img / 255.0
    # Return tensor
    return img

#### Steps

1. Load a few random images
2. Make predictions on loaded images
3. Plot the original image(s) along with the model's predictions, prediction probability and truth label

In [ ]:
import os
import random

plt.figure(figsize=(17, 10))
for i in range(3):
    class_names = list(class_f1_scores.keys())
    # Choose random image(s) from random class(es)
    class_name = random.choice(class_names)
    filename = random.choice(os.listdir(os.path.join(test_dir, class_name)))
    filepath = os.path.join(test_dir, class_name, filename)
    
    # Load and prepare image
    img = load_and_prep_images(filepath, scale=False)
    img_expanded = tf.expand_dims(img, axis=0)
    pred_prob = model.predict(img_expanded) # get prediction probability
    pred_prob_max = pred_prob.argmax()
    pred_class = class_names[pred_prob_max] # get highest prediction probability index

    # Plot image(s)
    plt.subplot(1, 3, i+1)
    plt.imshow(img/255.)

    if class_name == pred_class: # if predicted class matches truth class, make text green, else red
        title_color = "g"
    else:
        title_color = "r"
    plt.title(f"Actual: {class_name} | Pred: {pred_class} | Prob: {pred_prob.max():.2f}", color=title_color)
    plt.axis(False)

## Finding the most incorrect predictions

To find out where our model is the most incorrect:
1. Get all of the image file paths in the test dataset using `list_files()` method
2. Create a pandas DataFrame of the image filepaths, ground truth labels, predicted classes (from our model), max prediction probabilities, prediction class names, ground truth class names.
3. Use the DataFrame to find all the wrong predictions (where the ground truth label doesn't match the prediction).
4. Sort the DataFrame based on wrong predictions (descending).
5. Visualize the images with the highest prediction probabilities but have the wrong prediction.

In [ ]:
# 1. Get all the img filepaths
import tensorflow as tf

filepaths = []
for filepath in test_data.list_files(os.path.join(test_dir, "*/*.jpg"), shuffle=False):
    filepaths.append(filepath.numpy())

filepaths[:10]

In [ ]:
# 2. Create a DataFrame of different params for each of our test images
import pandas as pd

pred_df = pd.DataFrame({"img_path":filepaths,
                        "y_true":y_labels,
                        "y_pred":pred_classes,
                        "pred_conf":preds_probs.max(axis=1), # get max pred prob
                        "y_true_classname":[class_names[i] for i in y_labels],
                        "y_pred_classname":[class_names[i] for i in pred_classes]})
pred_df


In [ ]:
# 3. Find out which predictions are wrong
pred_df["pred_correct"] = pred_df["y_true"] == pred_df["y_pred"]
pred_df.head()

In [ ]:
# 4. Sort our DataFrame to have most wrong predictions at the top
top_100_wrong = pred_df[pred_df["pred_correct"]==False].sort_values("pred_conf", ascending=False).head(100)
top_100_wrong

In [ ]:
# 5. Visualize the test data samples which have the wrong prediction but highest prediction probability
images_to_view = 9
start_index = 20
plt.figure(figsize=(15,10))
for i, row in enumerate(top_100_wrong[start_index:start_index+images_to_view].itertuples()):
    print(i)
    plt.subplot(3, 3, i+1)
    expanded_img = load_and_prep_images(row[1], scale=False)/255.
    plt.imshow(expanded_img)
    plt.title(f"Actual: {row.y_true_classname} \nPred: {row.y_pred_classname} \nProb: {row.pred_conf:.2f}", color="r")
    plt.axis(False)
    


### Test out on custom images

In [ ]:
# Get custom images
!wget -nc -P ../Downloads/ https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

# Unzip
unzip_data('../Downloads/custom_food_images.zip', '../Downloads')

# Check number of images and subdirectories in the dataset
walk_through_dir('../Downloads/custom_food_images')

In [ ]:
# Get the images filepaths
custom_food_images = ['../Downloads/custom_food_images/' + img_path for img_path in os.listdir("../Downloads/custom_food_images")]
custom_food_images

In [ ]:
# Make predictions on custom food images
for img in custom_food_images:
    img = load_and_prep_images(img, scale=False) # load in target image and turn it into tensor
    pred_prob = model.predict(tf.expand_dims(img, axis=0)) # make prediction on image with shape [None, 224, 224, 3]
    pred_class = class_names[pred_prob.argmax()] # find the predicted class label
    # Plot the image with appropriate annotations
    plt.figure()
    plt.imshow(img/255.) # imshow() requires float inputs to be normalized
    plt.title(f"pred: {pred_class}, prob: {pred_prob.max():.2f}")
    plt.axis(False)
    
